<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/2021m1/035%E8%A4%87%E9%9B%91%E3%81%AA%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E5%BD%A2%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Microsoft Wordのdocxファイルを操作してみます。

まず、外部ライブラリをインストールします。インストールは、Colabを開いたあと一度だけ実施します。(こんなのどうやって見付けるんだ、と思うかもしれませんが、「word docx python」でGoogle検索しているだけです。)

In [ ]:
# https://gammasoft.jp/support/how-to-use-python-docx-for-word-file/

# Google Colabの場合のみ。(Localにはインストール済み?)
!pip install python-docx

ウェブページの解説に従って、サンプルファイルを読んでみます。 http://www.jsgi.org/doc/Layout_Sample.docx

doc変数には、docファイルの内容全体が含まれていますが、たぶん複雑なデータ構造なので、そのままprint()で表示したりすることはできません。

In [ ]:
import docx
doc = docx.Document("Layout_Sample.docx")


そのままセーブすると、コピーが作れます。

In [ ]:
doc.save("Sample2.docx")

文書は段落に分割されているようです。リストdoc.pragraphsには、それぞれの段落の内容が含まれていて、段落の.text属性に、素の文章が入っています。

In [ ]:
for para in doc.paragraphs:
    print(para.text)

段落の切れ目がどうなっているのかよくわからないので、行番号を付けてみます。enumerateを使います。

In [ ]:
for i, para in enumerate(doc.paragraphs):
    print(i, para.text)

Google Translateで翻訳してみましょう。

これも外部ライブラリを見付けてきました。

In [ ]:
# https://pypi.org/project/googletrans/

# Google Colabのみ。

!pip install googletrans==3.1.0a0

基本的な使い方はこんな感じです。Translatorクラスをインポートし、translatorオブジェクトを生成し、それを使って文章を翻訳します。

In [ ]:
from googletrans import Translator
translator = Translator()
translator.translate("Happy New Year!", dest='ja').text

ドイツ語でも大丈夫。

In [ ]:
from googletrans import Translator
translator = Translator()
translator.translate("Guten morgen!", dest='en').text

docxを読みこみ、段落ごとに翻訳します。

In [ ]:
from googletrans import Translator
translator = Translator()

import docx
# doc = docx.Document("/content/drive/MyDrive/Sample.docx")
doc = docx.Document("Sample2.docx")

for para in doc.paragraphs:
    translated = translator.translate(para.text, dest='en')
    print(translated.origin)
    print(translated.text)

翻訳結果を、別のdocxファイルに保存します。

In [ ]:
from googletrans import Translator
translator = Translator()
for para in doc.paragraphs:
    translated = translator.translate(para.text, dest='en')
    para.text = translated.text
# doc.save("/content/drive/MyDrive/Translated.docx")
doc.save("Translated.docx")

こんどはPDFから文章を抽出してみます。

これも検索で見付けてきました。

In [ ]:
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file
!pip install textract

Sample.pdfのなかの文章を抽出します。

In [ ]:
import textract
# text = textract.process("/content/drive/MyDrive/Sample.pdf")
text = textract.process("Sample.pdf")
text

先頭に"b"がついているのは、抽出した文字列が、Pythonによって文字と認識されておらず、ただのデータ列だと思われているという意味です。

文字列として認識してもらうには、文字エンコーディングを指定する必要があります。最近の文書なら、ほとんどの場合unicode (utf-8)でしょうから、それを指示します。

In [ ]:
import textract
# text = textract.process("/content/drive/MyDrive/Sample.pdf").decode('utf-8')
text = textract.process("Sample.pdf").decode('utf-8')
text

改行文字`\n`がランダムに入ってしまうので、これをとりのぞきます。

`replace()`という関数を使うとよいようです。

In [ ]:
import string
lines = text.replace('\n', '')
lines

行ごとに翻訳してみます。

In [ ]:
from googletrans import Translator
translator = Translator()

translated = translator.translate(lines, dest='en')
translated.text

残念ながら、google translateをPython経由で頻繁に呼びだすと、翻訳してくれなくなるようです。大量に翻訳したい場合は、やはりお金を払うサービスでないとむりかもしれません。

次の例では、google translateの代わりに、DeepL APIを利用しています。(API=Application Programming Interface; 外部のプログラムを呼びだして利用するための手順)

DeepL APIは有料です。ここでは、松本のアカウントを使って翻訳してみます。

auth_keyの部分に、あらかじめDeepLから提供された特殊な文字列を書く必要があります。ここに書いてしまうと、githubを通じて世界中から見えてしまうので、auth_keyもまた別の方法でみなさんに貸しだします。

In [ ]:
# https://michimani.net/post/programming-using-deepl-api-by-python/
import urllib.parse
import urllib.request
import json

DEEPL_TRANSLATE_EP = 'https://api.deepl.com/v2/translate'

def translate(text, s_lang='', t_lang='JA'):
    # DeepL APIはウェブ上で働くAPIなので、呼びだすにはHTTPプロトコルでデータの
    # やりとりをする必要があります。この関数はその煩雑な手順をパッケージしたも
    # のです。
    # s_langは元の文章の言語、t_langは翻訳後の言語で、デフォルト値がJAなので、
    # 指定しない場合は日本語に翻訳されます。

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded; utf-8'
    }

    params = {
        'auth_key': "",
        'text': text,
        'target_lang': t_lang
    }

    if s_lang != '':
        params['source_lang'] = s_lang

    req = urllib.request.Request(
        DEEPL_TRANSLATE_EP,
        method='POST',
        data=urllib.parse.urlencode(params).encode('utf-8'),
        headers=headers
    )

    try:
        with urllib.request.urlopen(req) as res:
            res = json.loads(res.read().decode('utf-8'))
            return res["translations"][0]["text"]
    except urllib.error.HTTPError as e:
        print(e)

translate("まず、外部ライブラリをインストールします。インストールは、Colabを開いたあと一度だけ実施します。(こんなのどうやって見付けるんだ、と思うかもしれませんが、「word docx python」でGoogle検索しているだけです。)", t_lang="en")

In [ ]:
import docx
# doc = docx.Document("/content/drive/MyDrive/Sample.docx")
doc = docx.Document("Sample2.docx")

for para in doc.paragraphs:
    translated = translate(para.text, t_lang="en")
    print(translated)

こんな風に、docやpdfを読みだすモジュールと、翻訳サービスをpythonで連結することで、手作業でしかできなかったことを自動処理にできます。一種のRPA(Robotic Process Automation)ですね。

 ExcelのファイルxlsxもPythonから操作できますが、その話はまた後ほど機会があれば説明します。